# **지능형 IoT 응용**
#### 한국기술교육대학교 컴퓨터공학부 스마트 IoT 트랙


---

# 07. 마이크로컨트롤러용 호출어 감지 애플리케이션


---

### Acknowledgement


이 자료는 다음 서적의 내용을 바탕으로 작성되었음
- 초소형 머신러닝 TinyML, 피트 워든, 대니얼 시투나야케 지음, 맹윤호, 임지순 옮김, 한빛미디어
 - 7장

---

### 왜 호출어 감지 애플리케이션이 필요할까?

- 여러 가지 음성 인식 비서 제품이 등장
  - Google Assistant (https://assistant.google.com/)
  - Apple Siri (https://www.apple.com/kr/siri/)
  - Amazon Alexa (https://developer.amazon.com/en-US/alexa)
  - Samsung Bixby (https://www.samsung.com/sec/apps/bixby/)
  - SKT NUGU (https://www.nugu.co.kr/)

- 음성 인식 서비스는 대부분의 스마트폰에 내장되어 있고, 다양한 스마트 스피커 제품으로도 나와 있음

- 이러한 음성 인식 서비스는 대규모의 머신러닝 모델을 실행하는 서버에 의해 구동됨
  - 서버에서 음성 인식, 자연어 처리, 사용자 쿼리에 대한 응답 생성을 수행
  - 사용자가 질문을 하면 오디오 스트림으로 서버에 전송되고 서버는 의미를 파악하여 필요한 정보를 찾은 다음 적절한 응답을 보내는 것

- 음성 인식 비서는 사용자가 언제 어디에 있든 24시간 내내 목소리를 들을 수 있어야 함
  - 사용자가 거실에 앉아 있을 때나
  - 차를 타고 고속도로를 주행할 때나
  - 스마트폰을 들고 야외에서 대화를 할 때나

- 그런데 사용자가 언제 어디에서든 하는 말을 다 듣고 서버로 전송한다면?
  - 심각한 프라이버시 문제 발생 가능
  - 엄청난 네트워크 대역폭 사용
  - 막대한 연산량, 에너지 사용 (스마트폰 같은 장치의 배터리 급격한 소모)

- 음성 인식 비서가 듣고 처리해야 하는 말을 구별해주기 위해서 호출어(Hotword, Wake word, Wake-up command) 감지 방법을 사용
  - "Hey Google" / "OK Google"
  - "Hey Siri"
  - "Alexa"
  - "Hi Bixby"
  - "아리아" (NUGU)

- 호출어를 인식하는 작은 모델을 훈련시켜 저전력 칩에서 실행할 수 있으며 이를 스마트폰이나 스마트 스피커 같은 디바이스에 내장하면 항상 호출어를 들을 수 있음
  - TinML이 중요한 역할을 할 수 있음
  - 불필요한 데이터를 서버에 전송하지 않고 개인정보보호, 효율성을 향상할 수 있는 방법  

## 7.1 호출어 감지 애플리케이션

- 음성 오디오를 분류하는 임베디드 애플리케이션
  - 음성 명령 데이터셋을 가지고 훈련된 18KB 모델을 사용
  - 'yes', 'no' 두 단어를 인식하고 알 수 없는 단어, 무음을 구별함
  - 마이크를 이용해 주변에서 발생하는 오디오를 듣고 인식 결과에 따라 LED를 켜거나 화면에 데이터를 표시

- 애플리케이션 예제 코드
  - TinyML 번역서의 한글 소스코드 저장소의 예제 코드
  - https://github.com/yunho0130/tensorflow-lite/tree/master/tensorflow/lite/micro/examples/micro_speech    


앞서 보았던 머신러닝 애플리케이션의 일반적인 작동 방식과 비슷한 흐름으로 동작하지만 호출어 감지 애플리케이션은 더 복잡함

- 머신러닝 애플리케이션의 일반적인 작업 흐름
  - 입력을 얻는다
  - 입력을 전처리해 모델에 공급하기 적합한 특징을 추출한다
  - 처리된 입력에 대한 추론을 실행한다
  - 모델의 출력을 후처리한다
  - 결과 정보를 사용하여 필요한 작업을 수행한다

- 이전에 본 hello world 예제는 각 단계가 매우 간단
  - 입력은 단순한 부동소수점 숫자
  - 특별한 전처리가 없고 특징 추출 단계가 없음
  - 특별한 후처리 없음

- 호출어 감지 애플리케이션이 복잡한 이유
  - 마이크에서 오디오 데이터를 입력으로 받음
  - 모델에 공급되기 전에 많은 전처리 필요
  - 모델은 분류기의 일종으로 각 클래스에 속할 확률을 출력하는데 이 결과를 파싱하고 이해 가능하도록 가공해야 함
  - 실시간 입력되는 라이브 데이터에 대해 지속적으로 추론을 수행해야 함. 추론의 흐름을 이해하도록 코드를 작성해야 함
  - 모델이 더 크고 복잡함

## 7.2 애플리케이션 아키텍처

<img src="./07.호출어_감지_애플리케이션/07-1.architecture.png" width="90%" height="90%">
<center>호출어 감지 애플리케이션 아키텍처</center>



#### 애플리케이션의 구성 요소

- 메인 루프
  - hello world 예제와 마찬가지로 호출어 감지 애플리케이션도 연속 루프로 실행됨
  - 모든 후속 프로세스는 루프 안에 포함되어 있으며 마이크로컨트롤러가 실행할 수 있는 속도로 계속 실행됨

- 오디오 추출기
  - 마이크에서 raw 오디오 데이터를 캡처함
  - 오디오 캡처 방법은 장치마다 다르기 때문에 이 모듈은 장치에 맞는 커스텀 버전으로 재정의할 수 있음

- 특징 추출기
  - raw 오디오 데이터를 모델에 필요한 스펙트로그램(spectrogram) 형식으로 변환함
  - 메인 루프의 일부로 인터프리터에 1초 간격의 오버랩이 있는 데이터 시퀀스를 제공함

- TF Lite 인터프리터
  - 텐서플로우 라이트 모델을 실행하여 입력 스펙트로그램을 확률 세트로 변환함
  - 여기서 확률 세트라는 것은 각 클래스(yes, no, 알 수 없음, 무음) 별 확률값의 모음을 의미

- 모델
  - 모델은 데이터 배열로 표현되며 인터프리터에 의해 실행됨

- 명령 인식기
  - 인터프리터의 출력 결과를 이용하여 호출어 음성을 인식함
  - 추론은 초당 여러 번 실행되므로 여러 개의 결과를 집계하고 알려진 단어(yes/no)가 들렸는지 평균적으로 결정함

- 명령 응답기
  - 음성 명령(호출어)이 인식되면 장치의 출력 기능을 통해 사용자에게 알림
  - 장치에 따라 LED를 깜박이거나 디스플레이에 데이터를 표시할 수 있음
  - 장치 유형에 따라 이 모듈을 재정의할 수 있음






#### 예제 모델 소개

- 모델 훈련 데이터셋 (Speech Commands)
  - 온라인 크라우드소싱 방식으로 수집된 30여개의 짧은 단어(1초 길이) 오디오 데이터 묶음으로 구성됨
  - 버전 1 (약 65000개), 버전 2 (약 106000개)
  - https://www.tensorflow.org/datasets/catalog/speech_commands
  - 관련 arXiv 논문 링크: https://arxiv.org/abs/1804.03209

- 7장 예제 모델의 출력
  - yes, no, unknown(알 수 없음), silence(무음) 네 가지 클래스를 분류할 수 있게 훈련됨
  - 1초의 오디오 데이터를 사용하여 네 개 클래스 중 하나를 나타낼 가능성을 예측하여 각 클래스에 하나씩 네 개의 확률을 출력함
  - 위 데이터셋을 사용하여 다른 단어를 인식하도록 모델을 훈련할 수도 있음 (8장에서 소개)

- 모델의 입력
  - 오디오 데이터의 Spectrogram(스펙트로그램)
  - raw 오디오 데이터를 전처리하여 spectrogram을 만든 후 이를 모델의 입력으로 사용
  - spectrogram 데이터는 2차원 배열로 나타낼 수 있기 때문에 2D 텐서(tensor)로 모델에 공급됨

- Spectrogram
  - 소리나 파동을 시각화하여 파악하기 위한 기법으로 파형(waveform)과 스펙트럼(spectrum)의 특징이 조합되어 있는 heat map 그래프로 표현됨
  - 파형(waveform)에서는 시간 축의 변화에 따른 진폭의 변화를 볼 수 있고 스펙트럼(spectrum)에서는 주파수 축의 변화에 따른 진폭의 변화를 볼 수 있는데, 스펙트로그램에서는 시간 축과 주파수 축의 변화에 따른 진폭의 차이를 한번에 확인할 수 있음
  - 위키백과 내용 참고 (https://ko.wikipedia.org/wiki/%EC%8A%A4%ED%8E%99%ED%8A%B8%EB%A1%9C%EA%B7%B8%EB%9E%A8)

- 스펙트로그램 예제

<img src="https://upload.wikimedia.org/wikipedia/commons/c/c5/Spectrogram-19thC.png">
<center>(이미지 출처: https://en.wikipedia.org/wiki/Spectrogram)</center>


<img src="https://miro.medium.com/max/1225/0*2xSeYn8Hghl8MTar.png">
<center>(이미지 출처: https://medium.com/@ODSC/deep-learning-for-speech-recognition-cbbebab15f0d)</center>


- 모델 신경망 아키텍처
  - CNN(Convolutional Neural Network)
  - CNN은 인접한 값 그룹 사이의 관계 정보가 포함된 다차원 텐서에 잘 작동하도록 설계된 네트워크로서 인접한 픽셀 그룹이 모양, 패턴, 질감을 나타낼 수 있는 이미지 데이터에 많이 사용







## 7.3 테스트 코드

#### 주요 코드
- micro_speech_test.cc : 입력 오디오 신호의 스펙트로그램 데이터에 대한 추론을 실행하고 결과를 해석하는 방법
- audio_provider_test.cc : 오디오 추출기를 사용하는 방법
- feature_provider_mock_test.cc : 데이터를 전달하기 위한 오디오 추출기의 모의(가짜) 구현으로 특징 추출기를 사용하는 방법
- recognize_commands_test.cc : 호출어 인식 여부를 결정하기 위해 모델 출력을 해석하는 방법
- command_responder_test.cc : 명령 응답기를 호출하여 출력을 트리거하는 방법

https://github.com/yunho0130/tensorflow-lite/tree/master/tensorflow/lite/micro/examples/micro_speech




### 7.3.1 기본 흐름 (micro_speech_test.cc)

- micro_speech_test.cc에서 모델 로드, 인터프리터 설정, 텐서 할당, 인터프리터 실행 등의 기본 동작은 이전 hello world 예제와 비슷함
  - https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/micro_speech/micro_speech_test.cc
- 주요 차이점
  - hello world 예제에서는 AllOpsResolver를 이용하여 모델을 실행하는 데 필요한 Op를 가져옴. 하지만 이 방법은 사용 가능한 모든 Op를 로드하게 되어 사용하지 않는 Op에 대한 메모리 자원이 낭비되는 단점이 있음
  - 따라서 모델 실행에 필요한 Op만 포함하도록 하기 위해 MicroMutableOpResolver를 사용
  - 스펙트로그램 데이터를 이용하여 호출어를 인식하는데 CNN 모델을 사용하므로 CNN 모델 실행에 필요한 Op(연산)만 로드하도록 되어 있음




```cpp
#include "tensorflow/lite/micro/examples/micro_speech/micro_features/no_micro_features_data.h"
#include "tensorflow/lite/micro/examples/micro_speech/micro_features/tiny_conv_micro_features_model_data.h"
#include "tensorflow/lite/micro/examples/micro_speech/micro_features/yes_micro_features_data.h"
#include "tensorflow/lite/micro/kernels/micro_ops.h"
#include "tensorflow/lite/micro/micro_error_reporter.h"
#include "tensorflow/lite/micro/micro_interpreter.h"
#include "tensorflow/lite/micro/micro_mutable_op_resolver.h"
#include "tensorflow/lite/micro/testing/micro_test.h"
#include "tensorflow/lite/schema/schema_generated.h"
#include "tensorflow/lite/version.h"
```

- tensorflow/lite/micro/examples/micro_speech/micro_features/no_micro_features_data.h
  - 테스트 용으로 사용하는 "no" 오디오 신호의 특징 데이터
- tensorflow/lite/micro/examples/micro_speech/micro_features/yes_micro_features_data.h
  - 테스트 용으로 사용하는 "yes" 오디오 신호의 특징 데이터



```cpp
TF_LITE_MICRO_TESTS_BEGIN

TF_LITE_MICRO_TEST(TestInvoke) {
  // Set up logging.
  tflite::MicroErrorReporter micro_error_reporter;
  tflite::ErrorReporter* error_reporter = &micro_error_reporter;

  // Map the model into a usable data structure. This doesn't involve any
  // copying or parsing, it's a very lightweight operation.
  const tflite::Model* model =
      ::tflite::GetModel(g_tiny_conv_micro_features_model_data);
  if (model->version() != TFLITE_SCHEMA_VERSION) {
    TF_LITE_REPORT_ERROR(error_reporter,
                         "Model provided is schema version %d not equal "
                         "to supported version %d.\n",
                         model->version(), TFLITE_SCHEMA_VERSION);
  }
```

- 로깅을 위한 MicroErrorReporter 객체 생성
- Model 객체 생성
  - 모델 데이터로는 tiny_conv_micro_features_model_data.h에 정의된 g_tiny_conv_micro_features_model_data 배열 이용

```cpp
  // tflite::ops::micro::AllOpsResolver resolver;
  tflite::MicroOpResolver<3> micro_op_resolver;
  micro_op_resolver.AddBuiltin(
      tflite::BuiltinOperator_DEPTHWISE_CONV_2D,
      tflite::ops::micro::Register_DEPTHWISE_CONV_2D());
  micro_op_resolver.AddBuiltin(tflite::BuiltinOperator_FULLY_CONNECTED,
                               tflite::ops::micro::Register_FULLY_CONNECTED());
  micro_op_resolver.AddBuiltin(tflite::BuiltinOperator_SOFTMAX,
                               tflite::ops::micro::Register_SOFTMAX());
```

- AllOpsResolver 객체 대신 MicroOpResolver 사용
  - 모델 실행에 필요한 operation만 추가해 줌
  - 2D convolution 연산을 위한 Op
  - Fully connected 연산을 위한 Op
  - Softmax 연산을 위한 Op

```cpp
  // Create an area of memory to use for input, output, and intermediate arrays.
  const int tensor_arena_size = 10 * 1024;
  uint8_t tensor_arena[tensor_arena_size];
```

- Tensor Arena 생성
  - 입력, 출력, 중간 생성 데이터 텐서를 저장할 메모리 공간

```cpp
  // Build an interpreter to run the model with.
  tflite::MicroInterpreter interpreter(model, micro_op_resolver, tensor_arena,
                                       tensor_arena_size, error_reporter);
  interpreter.AllocateTensors();
```

- 모델 실행을 위한 인터프리터 객체 생성
- 작업 메모리 할당

```cpp
  // Get information about the memory area to use for the model's input.
  TfLiteTensor* input = interpreter.input(0);

  // Make sure the input has the properties we expect.
  TF_LITE_MICRO_EXPECT_NE(nullptr, input);
  TF_LITE_MICRO_EXPECT_EQ(4, input->dims->size);
  TF_LITE_MICRO_EXPECT_EQ(1, input->dims->data[0]);
  TF_LITE_MICRO_EXPECT_EQ(49, input->dims->data[1]);
  TF_LITE_MICRO_EXPECT_EQ(40, input->dims->data[2]);
  TF_LITE_MICRO_EXPECT_EQ(1, input->dims->data[3]);
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteUInt8, input->type);
```

- 입력 데이터에 대한 포인터를 이용해 입력 텐서가 올바른 형태인지 확인
  - null 포인터가 아니어야 함
  - 입력 텐서의 차원은 4
  - 첫번째 차원(data[0])은 단일 요소를 포함하는 래퍼
  - 두번재 차원(data[1]), 세번째 차원(data[2]), 네번째 차원(data[3]은 스펙트로그램 데이터의 크기 (채널 하나인 2차원 데이터: 49행 40열 - 1960개 원소)
  - 데이터 타입은 8비트 integer (kTfLiteInt8)

```cpp
  // Copy a spectrogram created from a .wav audio file of someone saying "Yes",
  // into the memory area used for the input.
  const uint8_t* yes_features_data = g_yes_micro_f2e59fea_nohash_1_data;
  for (int i = 0; i < input->bytes; ++i) {
    input->data.uint8[i] = yes_features_data[i];
  }
```

- 테스트 추론을 위해 "yes"라고 말하는 오디오 신호에서 추출한 특징 데이터를 입력 텐서에 복사
  - micro_features/yes_micro_features_data.cc에 정의된 배열인 g_yes_micro_f2e59fea_nohash_1_data 이용


```cpp
  // Run the model on this input and make sure it succeeds.
  TfLiteStatus invoke_status = interpreter.Invoke();
  if (invoke_status != kTfLiteOk) {
    TF_LITE_REPORT_ERROR(error_reporter, "Invoke failed\n");
  }
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, invoke_status);
```

- 인터프리터를 실행하고 에러가 없는지 확인

```cpp
  // Get the output from the model, and make sure it's the expected size and
  // type.
  TfLiteTensor* output = interpreter.output(0);
  TF_LITE_MICRO_EXPECT_EQ(2, output->dims->size);
  TF_LITE_MICRO_EXPECT_EQ(1, output->dims->data[0]);
  TF_LITE_MICRO_EXPECT_EQ(4, output->dims->data[1]);
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteUInt8, output->type);
```

- 출력 텐서가 올바른 형태인지 확인
  - 출력 텐서의 차원은 2
  - 첫번째 차원 (data[0])은 단일 요소를 포함하는 래퍼
  - 두번재 차원 (data[1])은 모델 추론 결과를 저장 : 4가지 가능한 클래스(yes, no, unknown, silence) 각각의 확률
  - 데이터 타입은 8비트 integer


```cpp
  // There are four possible classes in the output, each with a score.
  const int kSilenceIndex = 0;
  const int kUnknownIndex = 1;
  const int kYesIndex = 2;
  const int kNoIndex = 3;

  // Make sure that the expected "Yes" score is higher than the other classes.
  uint8_t silence_score = output->data.uint8[kSilenceIndex];
  uint8_t unknown_score = output->data.uint8[kUnknownIndex];
  uint8_t yes_score = output->data.uint8[kYesIndex];
  uint8_t no_score = output->data.uint8[kNoIndex];
  TF_LITE_MICRO_EXPECT_GT(yes_score, silence_score);
  TF_LITE_MICRO_EXPECT_GT(yes_score, unknown_score);
  TF_LITE_MICRO_EXPECT_GT(yes_score, no_score);
```

- 출력 텐서 데이터에서 silence, unknown, yes, no 각 클래스의 확률값을 읽음
- yes의 확률이 다른 클래스의 확률보다 큰지 확인

```cpp
  // Now test with a different input, from a recording of "No".
  const uint8_t* no_features_data = g_no_micro_f9643d42_nohash_4_data;
  for (int i = 0; i < input->bytes; ++i) {
    input->data.uint8[i] = no_features_data[i];
  }

  // Run the model on this "No" input.
  invoke_status = interpreter.Invoke();
  if (invoke_status != kTfLiteOk) {
    TF_LITE_REPORT_ERROR(error_reporter, "Invoke failed\n");
  }
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, invoke_status);

  // Get the output from the model, and make sure it's the expected size and
  // type.
  output = interpreter.output(0);
  TF_LITE_MICRO_EXPECT_EQ(2, output->dims->size);
  TF_LITE_MICRO_EXPECT_EQ(1, output->dims->data[0]);
  TF_LITE_MICRO_EXPECT_EQ(4, output->dims->data[1]);
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteUInt8, output->type);

  // Make sure that the expected "No" score is higher than the other classes.
  silence_score = output->data.uint8[kSilenceIndex];
  unknown_score = output->data.uint8[kUnknownIndex];
  yes_score = output->data.uint8[kYesIndex];
  no_score = output->data.uint8[kNoIndex];
  TF_LITE_MICRO_EXPECT_GT(no_score, silence_score);
  TF_LITE_MICRO_EXPECT_GT(no_score, unknown_score);
  TF_LITE_MICRO_EXPECT_GT(no_score, yes_score);
```

- "no" 오디오 데이터에 대한 테스트 추론 및 결과 확인
- 입력 데이터
  - micro_features/no_micro_features_data.cc에 정의된 배열인 g_no_micro_f9643d42_nohash_4_data 이용
- 인터프리터 실행
- 출력 텐서 확인
- 각 클래스의 확률값을 읽고 no의 확률이 다른 것보다 큰지 확인

#### 테스트 실행

In [2]:
%cd /content
%rm -rf tensorflow
! git clone https://github.com/yunho0130/tensorflow-lite

/content
Cloning into 'tensorflow-lite'...
remote: Enumerating objects: 847199, done.
remote: Counting objects: 100% (187727/187727), done.
remote: Compressing objects: 100% (9942/9942), done.
remote: Total 847199 (delta 181888), reused 177785 (delta 177785), pack-reused 659472
Receiving objects: 100% (847199/847199), 580.16 MiB | 23.01 MiB/s, done.
Resolving deltas: 100% (685681/685681), done.
Updating files: 100% (19934/19934), done.


In [4]:
%cd /content/tensorflow-lite/

! make -f tensorflow/lite/micro/tools/make/Makefile test_micro_speech_test

/content/tensorflow-lite
tensorflow/lite/micro/tools/make/download_and_extract.sh "https://github.com/google/gemmlowp/archive/719139ce755a0f31cbf1c37f7f98adcc7fc9f425.zip" "7e8191b24853d75de2af87622ad293ba" tensorflow/lite/micro/tools/make/downloads/gemmlowp  
downloading https://github.com/google/gemmlowp/archive/719139ce755a0f31cbf1c37f7f98adcc7fc9f425.zip
tensorflow/lite/micro/tools/make/download_and_extract.sh "https://storage.googleapis.com/mirror.tensorflow.org/github.com/google/flatbuffers/archive/v1.11.0.tar.gz" "02c64880acb89dbd57eebacfd67200d8" tensorflow/lite/micro/tools/make/downloads/flatbuffers  
downloading https://storage.googleapis.com/mirror.tensorflow.org/github.com/google/flatbuffers/archive/v1.11.0.tar.gz
tensorflow/lite/micro/tools/make/download_and_extract.sh "https://github.com/mborgerding/kissfft/archive/v130.zip" "438ba1fef5783cc5f5f201395cc477ca" tensorflow/lite/micro/tools/make/downloads/kissfft patch_kissfft 
downloading https://github.com/mborgerding/kissf

### 7.3.2 오디오 추출기 (audio_provider.h / audio_provider.cc)

- 오디오 추출기는 장치의 마이크 하드웨어를 코드와 연결하는 역할을 수행
  - 장치마다 오디오 캡처를 위한 메커니즘이 모두 다르기 때문에 audio_provider.h는 오디오 데이터 요청을 위한 인터페이스를 정의하며 개발자는 지원하고자 하는 플랫폼에 대한 구현을 작성함

- 오디오 추출기의 핵심은 GetAudioSamples() 함수
  - https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/micro_speech/audio_provider.h


```cpp
TfLiteStatus GetAudioSamples(tflite::ErrorReporter* error_reporter,
                             int start_ms, int duration_ms,
                             int* audio_samples_size, int16_t** audio_samples);
```

- 16비트 PCM(펄스 코드 변조) 오디오 데이터의 배열을 반환
- 4개의 매개변수
  - ErrorReporter 객체
  - 시작 시간 (start_ms)
  - 기간 (duration_ms)
  - 오디오 샘플 크기 (audio_samples_size)
  - 오디오 샘플 데이터 (audio_samples)

#### 오디오 추출기 테스트

- 오디오 추출기를 사용하는 방법
 - audio_provider_test.cc에 있는 2가지 테스트 중 첫번째 테스트를 확인

```cpp
TF_LITE_MICRO_TEST(TestAudioProvider) {
  tflite::MicroErrorReporter micro_error_reporter;
  tflite::ErrorReporter* error_reporter = &micro_error_reporter;

  int audio_samples_size = 0;
  int16_t* audio_samples = nullptr;
  TfLiteStatus get_status =
      GetAudioSamples(error_reporter, 0, kFeatureSliceDurationMs,
                      &audio_samples_size, &audio_samples);
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, get_status);
  TF_LITE_MICRO_EXPECT_LE(audio_samples_size, kMaxAudioSampleSize);
  TF_LITE_MICRO_EXPECT_NE(audio_samples, nullptr);

  // Make sure we can read all of the returned memory locations.
  int total = 0;
  for (int i = 0; i < audio_samples_size; ++i) {
    total += audio_samples[i];
  }
}
```

- GetAudioSamples 함수를 호출하기 위해 필요한 객체 및 변수를 선언하고 호출
- 올바른 호출 결과를 얻었는지 확인
- 반환된 audio_samples 데이터 배열에서 모든 값을 읽을 수 있는지 확인


In [5]:
! make -f tensorflow/lite/micro/tools/make/Makefile test_audio_provider_test

g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/micro_speech/audio_provider_test.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/micro_speech/audio_provider_test.o
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/micro_speech/micro_features/micro_model_settings.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/micro_speech/micro_features/micro_model_settings.

### 7.3.3 특징 추출기 (feature_provider.h / feature_provider.cc)

- 특징 추출기는 오디오 추출기로부터 얻은 원시(raw) 오디오 데이터를 모델에 공급할 수 있는 스펙트로그램으로 변환하는 역할 수행
  - 메인 루프 중에 호출됨


- 특징 추출기 정의
  - feature_provider.h : https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/micro_speech/feature_provider.h
 - PopulateFeatureData() 함수: 현재 오디오 데이터 입력에 대한 특징 데이터(스펙트로그램)를 계산

```cpp
// Binds itself to an area of memory intended to hold the input features for an
// audio-recognition neural network model, and fills that data area with the
// features representing the current audio input, for example from a microphone.
// The audio features themselves are a two-dimensional array, made up of
// horizontal slices representing the frequencies at one point in time, stacked
// on top of each other to form a spectrogram showing how those frequencies
// changed over time.
class FeatureProvider {
 public:
  // Create the provider, and bind it to an area of memory. This memory should
  // remain accessible for the lifetime of the provider object, since subsequent
  // calls will fill it with feature data. The provider does no memory
  // management of this data.
  FeatureProvider(int feature_size, uint8_t* feature_data);
  ~FeatureProvider();

  // Fills the feature data with information from audio inputs, and returns how
  // many feature slices were updated.
  TfLiteStatus PopulateFeatureData(tflite::ErrorReporter* error_reporter,
                                   int32_t last_time_in_ms, int32_t time_in_ms,
                                   int* how_many_new_slices);

 private:
  int feature_size_;
  uint8_t* feature_data_;
  // Make sure we don't try to use cached information if this is the first call
  // into the provider.
  bool is_first_run_;
};
```


#### 특징 추출기 테스트

- 특징 추출기 사용 방법은 feature_provider_mock_test.cc의 테스트 코드에서 확인할 수 있음
  - https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/micro_speech/feature_provider_mock_test.cc

- 특징 추출을 위해서는 오디오 데이터가 필요하기 때문에 모의 오디오 추출기를 사용하여 오디오 데이터를 제공하도록 설정되어 있음
  - 모의 오디오 추출기: https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/micro_speech/audio_provider_mock.cc


```cpp
TF_LITE_MICRO_TEST(TestFeatureProviderMockYes) {
  tflite::MicroErrorReporter micro_error_reporter;
  tflite::ErrorReporter* error_reporter = &micro_error_reporter;

  uint8_t feature_data[kFeatureElementCount];
  FeatureProvider feature_provider(kFeatureElementCount, feature_data);

  int how_many_new_slices = 0;
  TfLiteStatus populate_status = feature_provider.PopulateFeatureData(
      error_reporter, /* last_time_in_ms= */ 0, /* time_in_ms= */ 970,
      &how_many_new_slices);
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, populate_status);
  TF_LITE_MICRO_EXPECT_EQ(kFeatureSliceCount, how_many_new_slices);

  for (int i = 0; i < kFeatureElementCount; ++i) {
    TF_LITE_MICRO_EXPECT_EQ(g_yes_micro_f2e59fea_nohash_1_data[i],
                            feature_data[i]);
  }
}
```

```cpp
  int8_t feature_data[kFeatureElementCount];
  FeatureProvider feature_provider(kFeatureElementCount, feature_data);
```

- FeatureProvider 객체를 생성하기 위해 생성자를 호출하여 kFeatureElementCount, feature_data 매개변수 인수 전달
  - kFeatureElementCount: 스펙트로그램에 있어야 하는 데이터 원소의 전체 수. 이는 모델을 학습할 때 결정된 것이고, micro_features/micro_model_settings.h에서 정의됨
 - https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/micro_speech/micro_features/micro_model_settings.h
 - feature_data: 스펙트로그램 데이터를 저장할 배열에 대한 포인터


```cpp
  TfLiteStatus populate_status = feature_provider.PopulateFeatureData(
      error_reporter, /* last_time_in_ms= */ 0, /* time_in_ms= */ 970,
      &how_many_new_slices);
```

- 지난 1초 동안의 오디오 특징 데이터(스펙트로그램)을 얻기 위해 PopulateFeatureData()가 호출됨

- 함수의 인수
  - ErrorReporter 객체
  - 함수가 마지막으로 호출된 시간을 나타내는 정수 (last_time_in_ms)
  - 현재 시간 (time_in_ms)
  - 새로운 특징 슬라이스 수(how_many_new_slices)를 계산하여 업데이트될 정수 변수에 대한 포인터. 슬라이스는 스펙트로그램의 한 행에 해당하며 일종의 시간 단위로 볼 수 있음

- 항상 가장 최근(마지막 순간)의 오디오 데이터가 필요하기 때문에 특징 추출기는 마지막으로 호출된 시간(last_time_in_ms)을 현재 시간(time_in_ms)과 비교함

- 마지막으로 호출된 시간과 현재 시간 사이에 캡처된 오디오에 대한 스펙트로그램 데이터를 생성하고 feature_data 배열을 업데이트하여 슬라이스를 추가한 후 1초보다 오래된 것은 삭제함

- PopulateFeatureData()를 실행하면 모의 오디오 추출기에 오디오를 요청함. 모의 오디오 추출기는 yes를 나타내는 오디오 데이터를 제공하고 특징 추출기는 이를 처리하여 결과를 제공함

```cpp
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, populate_status);
  TF_LITE_MICRO_EXPECT_EQ(kFeatureSliceCount, how_many_new_slices);

  for (int i = 0; i < kFeatureElementCount; ++i) {
    TF_LITE_MICRO_EXPECT_EQ(g_yes_micro_f2e59fea_nohash_1_data[i],
                            feature_data[i]);
  }
}
```

- 추출된 데이터에 에러가 없는지 확인
- 생성된 데이터를 모의 오디오 추출기가 제공한 yes 입력에 일치하는 스펙트로그램 데이터와 비교

```cpp
TF_LITE_MICRO_TEST(TestFeatureProviderMockNo) {
  tflite::MicroErrorReporter micro_error_reporter;
  tflite::ErrorReporter* error_reporter = &micro_error_reporter;

  uint8_t feature_data[kFeatureElementCount];
  FeatureProvider feature_provider(kFeatureElementCount, feature_data);

  int how_many_new_slices = 0;
  TfLiteStatus populate_status = feature_provider.PopulateFeatureData(
      error_reporter, /* last_time_in_ms= */ 4000, /* time_in_ms= */ 4970,
      &how_many_new_slices);
  TF_LITE_MICRO_EXPECT_EQ(kTfLiteOk, populate_status);
  TF_LITE_MICRO_EXPECT_EQ(kFeatureSliceCount, how_many_new_slices);

  for (int i = 0; i < kFeatureElementCount; ++i) {
    TF_LITE_MICRO_EXPECT_EQ(g_no_micro_f9643d42_nohash_4_data[i],
                            feature_data[i]);
  }
}
```

- 두번째 테스트는 "no" 오디오에 대한 특징 추출기 테스트
  - last_time_in_ms와 time_in_ms 값이 달라짐

- 테스트 실행


In [7]:
%cd /content/tensorflow-lite/

! make -f tensorflow/lite/micro/tools/make/Makefile test_feature_provider_mock_test

/content/tensorflow-lite
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/micro_speech/feature_provider_test.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/micro_speech/feature_provider_test.o
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/micro_speech/audio_provider_mock.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/micro_speech/audio_provider_mock.o
g

#### 특징 추출기의 스펙트로그램 생성 방법

- 특징 추출기 구현
  - feature_provider.cc : https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/micro_speech/feature_provider.cc

- 특징 추출기의 역할
  - 1초 오디오의 스펙트로그램을 나타내는 배열을 채우는 것
  - 루프에서 반복 호출되도록 설계됐기 때문에 현재 호출과 직전 호출 사이의 시간 동안의 데이터에 대한 특징을 생성
  - 1초가 경과되기 전에 호출된 경우 이전 출력의 일부를 유지하고 누락된 부분의 특징만 생성

- 스펙트로그램은 40열 49행의 2D 배열로 표시됨
  - 각 행은 주파수 버킷 40개로 분할된 30ms 오디오 샘플의 특징을 나타냄
  - 각 행을 만들기 위해 30ms 오디오 입력 슬라이스에 FFT (Fast Fourier Transform, 고속 푸리에 변환) 알고리즘을 실행. FFT는 데이터 샘플에서 오디오 주파수 분포를 분석하고 각각 0에서 255사이 값을 갖는 256개의 주파수 버킷 배열을 생성함
  - 256개 주파수 버킷 배열은 6개 그룹으로 평균화 되어 버킷 40개를 생성함
  - 30ms 오디오 샘플 창을 20ms 간격으로 이동하며 전체 1초 샘플을 모두 처리할 때까지 위 연산을 수행



<img src="./07.호출어_감지_애플리케이션/07-2.spectrogram_feature.png" width="90%" height="90%">
<center>오디오 샘플 데이터에서 스펙트로그램을 구하는 과정</center>

- feature_provider.cc의 PopulateFeatureData() 주요 코드

```cpp
  // Quantize the time into steps as long as each window stride, so we can
  // figure out which audio data we need to fetch.
  const int last_step = (last_time_in_ms / kFeatureSliceStrideMs);
  const int current_step = (time_in_ms / kFeatureSliceStrideMs);

  int slices_needed = current_step - last_step;
```

- PopulateFeatureData()가 마지막으로 호출된 시간을 기준으로 실제로 생성해야 하는 슬라이스를 결정

```cpp
  // If this is the first call, make sure we don't use any cached information.
  if (is_first_run_) {
    TfLiteStatus init_status = InitializeMicroFeatures(error_reporter);
    if (init_status != kTfLiteOk) {
      return init_status;
    }
    is_first_run_ = false;
    slices_needed = kFeatureSliceCount;
  }
  if (slices_needed > kFeatureSliceCount) {
    slices_needed = kFeatureSliceCount;
  }
  *how_many_new_slices = slices_needed;
```

- 이전에 실행되지 않았거나 1초 이상 전에 실행된 경우 최대 슬라이스 수를 생성
  - 결과로 나오는 값(slices_needed)은 how_many_new_slices 변수에 기록



```cpp
  const int slices_to_keep = kFeatureSliceCount - slices_needed;
  const int slices_to_drop = kFeatureSliceCount - slices_to_keep;
  // If we can avoid recalculating some slices, just move the existing data
  // up in the spectrogram, to perform something like this:
  // last time = 80ms          current time = 120ms
  // +-----------+             +-----------+
  // | data@20ms |         --> | data@60ms |
  // +-----------+       --    +-----------+
  // | data@40ms |     --  --> | data@80ms |
  // +-----------+   --  --    +-----------+
  // | data@60ms | --  --      |  <empty>  |
  // +-----------+   --        +-----------+
  // | data@80ms | --          |  <empty>  |
  // +-----------+             +-----------+
  if (slices_to_keep > 0) {
    for (int dest_slice = 0; dest_slice < slices_to_keep; ++dest_slice) {
      int8_t* dest_slice_data =
          feature_data_ + (dest_slice * kFeatureSliceSize);
      const int src_slice = dest_slice + slices_to_drop;
      const int8_t* src_slice_data =
          feature_data_ + (src_slice * kFeatureSliceSize);
      for (int i = 0; i < kFeatureSliceSize; ++i) {
        dest_slice_data[i] = src_slice_data[i];
      }
    }
  }
```

- 기존 슬라이스 수를 계산하고 배열 데이터를 이동하여 새로운 슬라이스를 위한 공간을 만듦
  - slices_to_keep: 이전 슬라이스 중 유지할 슬라이스 수
  - slices_to_drop: 이전 슬라이스 중 버릴 슬라이스 수

```cpp
  // Any slices that need to be filled in with feature data have their
  // appropriate audio data pulled, and features calculated for that slice.
  if (slices_needed > 0) {
    for (int new_slice = slices_to_keep; new_slice < kFeatureSliceCount;
         ++new_slice) {
      const int new_step = (current_step - kFeatureSliceCount + 1) + new_slice;
      const int32_t slice_start_ms = (new_step * kFeatureSliceStrideMs);
      int16_t* audio_samples = nullptr;
      int audio_samples_size = 0;
      // TODO(petewarden): Fix bug that leads to non-zero slice_start_ms
      GetAudioSamples(error_reporter, (slice_start_ms > 0 ? slice_start_ms : 0),
                      kFeatureSliceDurationMs, &audio_samples_size,
                      &audio_samples);
      if (audio_samples_size < kMaxAudioSampleSize) {
        TF_LITE_REPORT_ERROR(error_reporter,
                             "Audio data size %d too small, want %d",
                             audio_samples_size, kMaxAudioSampleSize);
        return kTfLiteError;
      }
```

- 새로 만들어야 하는 슬라이스마다 한 번씩 반복되는 루프를 시작
  - GetAudioSamples() 함수를 사용하여 오디오 추출기에 해당 슬라이스에 대한 오디오 샘플을 획득


```cpp
      uint8_t* new_slice_data = feature_data_ + (new_slice * kFeatureSliceSize);
      size_t num_samples_read;
      TfLiteStatus generate_status = GenerateMicroFeatures(
          error_reporter, audio_samples, audio_samples_size, kFeatureSliceSize,
          new_slice_data, &num_samples_read);
      if (generate_status != kTfLiteOk) {
        return generate_status;
      }
    }
```

- 오디오 샘플 데이터는 GenerateMicroFeatures() 함수를 이용하여 해당 슬라이스 오디오에 대한 스펙트로그램 정보를 반환
  - micro_features/micro_features_generator.h에 정의됨


- 스펙트로그램 데이터가 준비되면 모델로 추론을 수행할 수 있음. 추론을 마치고 나면 결과를 해석해야 하는데 이것은 명령 인식기에서 진행됨


### 7.3.4 명령 인식기 (recognize_commands.h / recognize_commands.cc)

- RecognizeCommands 클래스
  - 모델 추론이 실행된 후 학습된 단어가 사용됐는지 추론 결과 확률 셋이 나오면 이것이 성공적인 호출어 감지를 의미하는지 여부를 판별하는 역할
  - https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/micro_speech/recognize_commands.h


- 입력 데이터에 대해 어떤 한 호출어 클래스의 확률이 특정 임계값 이상이면 해당 호출어가 쓰인 것으로 볼 수 있지만 실제 상황에서는 연속적인 오디오 데이터 입력에 대해 추론이 반복적으로 수행되므로 고려해야 할 점이 있음

- noted를 발음하고 이를 인식하는 경우의 예

<img src="./07.호출어_감지_애플리케이션/07-3.waveform.png" width="60%" height="60%">
<center>noted를 발음한 파형을 1초 윈도우로 캡처한 모습</center>

- 모델은 no라는 단어를 감지하도록 훈련됐으며 noted가 no와 다르다는 것은 구분할 수 있음

- 위 그림에서 갈색으로 표시된 1초 윈도우에 대해서 추론을 수행하면 이를 no로 분류할 가능성은 낮을 것임

- 하지만 검은색으로 표시된 1초 윈도우에 대해서 추론을 수행하면 noted의 첫 음절 부분만 해당되고 이는 no와 같기 때문에 모델은 이것이 no일 확률이 높은 것으로 해석할 수 있음

- 이와 같은 문제 때문에 하나의 추론 결과에만 의존하여 호출어의 유무를 판단해서는 안 됨



- RecognizeCommands 명령 인식기
  - 몇 개의 연속적인 추론에 대한 각 단어의 평균 점수를 계산하고 어떤 한 단어로 판단하기에 점수가 충분히 높은지 확인하는 과정을 통해 최종 결과를 냄
  - 이를 위해 추론 결과가 나올 때마다 이 결과를 인식기로 전달함

- recognize_commands.h의 RecognizeCommands 클래스 정의

```cpp
class RecognizeCommands {
 public:
  // labels should be a list of the strings associated with each one-hot score.
  // The window duration controls the smoothing. Longer durations will give a
  // higher confidence that the results are correct, but may miss some commands.
  // The detection threshold has a similar effect, with high values increasing
  // the precision at the cost of recall. The minimum count controls how many
  // results need to be in the averaging window before it's seen as a reliable
  // average. This prevents erroneous results when the averaging window is
  // initially being populated for example. The suppression argument disables
  // further recognitions for a set time after one has been triggered, which can
  // help reduce spurious recognitions.
  explicit RecognizeCommands(tflite::ErrorReporter* error_reporter,
                             int32_t average_window_duration_ms = 1000,
                             uint8_t detection_threshold = 200,
                             int32_t suppression_ms = 1500,
                             int32_t minimum_count = 3);

  // Call this with the results of running a model on sample data.
  TfLiteStatus ProcessLatestResults(const TfLiteTensor* latest_results,
                                    const int32_t current_time_ms,
                                    const char** found_command, uint8_t* score,
                                    bool* is_new_command);
```

- 클래스 생성자 기본값 정의
  - 평균화 창의 길이: average_window_duration_ms (1000)
  - 명령(호출어) 탐지의 기준이 되는 최소 평균 점수: detection_threshold (200)
  - 명령을 인식한 후 두 번째 명령을 인식하기 전에 기다리는 시간: suppression_ms (1500)
  - 결과를 세는 데 필요한 최소 추론 횟수: minimum_count (3)


- ProcessLatestResults() 메소드
  - 추론 결과를 이용하여 명령(호출어)를 판단하는 역할
  - 모델 출력을 포함하는 TfLiteTensor에 대한 포인터: latest_results
  - 현재 시간: current_time_ms
  - 감지한 명령의 이름: found_command
  - 명령의 평균 점수: score
  - 명령이 이전과 다른 새로운 것인지 같은 것인지 여부: is_new_command
  - 마지막 3개는 ProcessLatestResults() 함수 호출 결과를 전달하기 위한 용도로 사용되는 포인터 변수

- ProcessLatestResults() 메소드 구현
  - https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/micro_speech/recognize_commands.cc


```cpp
TfLiteStatus RecognizeCommands::ProcessLatestResults(
    const TfLiteTensor* latest_results, const int32_t current_time_ms,
    const char** found_command, uint8_t* score, bool* is_new_command) {
  if ((latest_results->dims->size != 2) ||
      (latest_results->dims->data[0] != 1) ||
      (latest_results->dims->data[1] != kCategoryCount)) {
    TF_LITE_REPORT_ERROR(
        error_reporter_,
        "The results for recognition should contain %d elements, but there are "
        "%d in an %d-dimensional shape",
        kCategoryCount, latest_results->dims->data[1],
        latest_results->dims->size);
    return kTfLiteError;
  }

  if (latest_results->type != kTfLiteUInt8) {
    TF_LITE_REPORT_ERROR(
        error_reporter_,
        "The results for recognition should be uint8 elements, but are %d",
        latest_results->type);
    return kTfLiteError;
  }
```

- 추론 결과를 담고 있는 텐서(latest_results)가 올바른 형태와 타입을 갖고 있는지 확인

```cpp
  if ((!previous_results_.empty()) &&
      (current_time_ms < previous_results_.front().time_)) {
    TF_LITE_REPORT_ERROR(
        error_reporter_,
        "Results must be fed in increasing time order, but received a "
        "timestamp of %d that was earlier than the previous one of %d",
        current_time_ms, previous_results_.front().time_);
    return kTfLiteError;
  }
```

- current_time_ms를 검사하여 평균화 창에서 가장 최근 결과를 낸 시간 이후인지 확인

```cpp
  // Add the latest results to the head of the queue.
  previous_results_.push_back({current_time_ms, latest_results->data.int8});

  // Prune any earlier results that are too old for the averaging window.
  const int64_t time_limit = current_time_ms - average_window_duration_ms_;
  
  while ((!previous_results_.empty()) &&
         previous_results_.front().time_ < time_limit) {
    previous_results_.pop_front();
  }
```

- 최신 추론 결과를 평균화할 결과의 목록에 추가

```cpp
  // If there are too few results, assume the result will be unreliable and
  // bail.
  const int64_t how_many_results = previous_results_.size();
  const int64_t earliest_time = previous_results_.front().time_;
  const int64_t samples_duration = current_time_ms - earliest_time;
  if ((how_many_results < minimum_count_) ||
      (samples_duration < (average_window_duration_ms_ / 4))) {
    *found_command = previous_top_label_;
    *score = 0;
    *is_new_command = false;
    return kTfLiteOk;
  }
```

- 평균화 창 내에 최소값(minimum_count 3)보다 적은 추론 결과만 있는 경우 유효한 평균을 제공할 수 없음
- 이 경우 가장 최근의 확률이 가장 높은 명령어이며, 점수는 0이고 명령이 새로운 명령이 아닌 것으로 처리


```cpp
  // Calculate the average score across all the results in the window.
  int32_t average_scores[kCategoryCount];

  for (int offset = 0; offset < previous_results_.size(); ++offset) {
    PreviousResultsQueue::Result previous_result =
        previous_results_.from_front(offset);

    const uint8_t* scores = previous_result.scores_;

    for (int i = 0; i < kCategoryCount; ++i) {
      if (offset == 0) {
        average_scores[i] = scores[i];
      } else {
        average_scores[i] += scores[i];
      }
    }
  }

  for (int i = 0; i < kCategoryCount; ++i) {
    average_scores[i] /= how_many_results;
  }
```

- 각 클래스 레이블(총 4개)의 점수의 평균을 계산
  - kCategoryCount: 4 (silence, unknown, yes, no)



```cpp
  // Find the current highest scoring category.
  int current_top_index = 0;
  int32_t current_top_score = 0;

  for (int i = 0; i < kCategoryCount; ++i) {
    if (average_scores[i] > current_top_score) {
      current_top_score = average_scores[i];
      current_top_index = i;
    }
  }

  const char* current_top_label = kCategoryLabels[current_top_index];
```

- 평균 점수가 가장 높은 클래스 레이블의 인덱스를 구하고 이를 이용해 해당 레이블 값을 획득
  - kCategoryLabels[kCategoryCount] 배열
  - https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/micro_speech/micro_features/micro_model_settings.cc


```cpp
  // If we've recently had another label trigger, assume one that occurs too
  // soon afterwards is a bad result.
  int64_t time_since_last_top;
  if ((previous_top_label_ == kCategoryLabels[0]) ||
      (previous_top_label_time_ == std::numeric_limits<int32_t>::min())) {
    time_since_last_top = std::numeric_limits<int32_t>::max();
  } else {
    time_since_last_top = current_time_ms - previous_top_label_time_;
  }

  if ((current_top_score > detection_threshold_) &&
      ((current_top_label != previous_top_label_) ||
       (time_since_last_top > suppression_ms_))) {
    previous_top_label_ = current_top_label;
    previous_top_label_time_ = current_time_ms;
    *is_new_command = true;
  } else {
    *is_new_command = false;
  }
  *found_command = current_top_label;
  *score = current_top_score;
```

- 결과가 유효한 탐지인지 확인
  - 점수가 사전에 정의된 임계값(200)보다 높고 마지막 유효한 탐지 후에 너무 빨리 발생하지 않았는지 확인

- 결과가 유효하면 is_new_command를 true로 설정
  - 함수 호출자가 실제로 명령이 감지되었는지 확인하기 위해 사용할 수 있는 변수


#### 명령 인식기 테스트

- 테스트 코드
  - https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/micro_speech/recognize_commands_test.cc



```cpp
TF_LITE_MICRO_TEST(RecognizeCommandsTestBasic) {
  tflite::MicroErrorReporter micro_error_reporter;
  tflite::ErrorReporter* error_reporter = &micro_error_reporter;

  RecognizeCommands recognize_commands(error_reporter);

  std::initializer_list<uint8_t> result_data = {255, 0, 0, 0};
  auto result_dims = {2, 1, 4};
  TfLiteTensor results = tflite::testing::CreateQuantizedTensor(
      result_data, tflite::testing::IntArrayFromInitializer(result_dims),
      "input_tensor", 0.0f, 128.0f);

  const char* found_command;
  uint8_t score;
  bool is_new_command;
  TF_LITE_MICRO_EXPECT_EQ(
      kTfLiteOk, recognize_commands.ProcessLatestResults(
                     &results, 0, &found_command, &score, &is_new_command));
}
```

 - RecognizeCommands 객체 생성
 - 테스트 용으로 사용할 추론 결과를 저장한 텐서 객체 생성
 - ProcessLatestResults() 함수의 호출 결과를 저장할 변수 선언
 - ProcessLatestResults() 함수 호출
 - 함수 반환값이 kTfLiteOk인지 확인

- 테스트 실행

In [9]:
%cd /content/tensorflow-lite/

! make -f tensorflow/lite/micro/tools/make/Makefile test_recognize_commands_test

/content/tensorflow-lite
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/micro_speech/recognize_commands_test.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/micro_speech/recognize_commands_test.o
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/micro_speech/recognize_commands.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/micro_speech/recognize_commands.o

### 7.3.5 명령 응답기 (command_responder.h / command_responder.cc)

- 명령 응답기 역할
  - 호출어가 감지됐음을 알려주는 출력을 생성함
  - RespondToCommand() 함수
  - https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/micro_speech/command_responder.h

- 명령 응답기는 각 유형의 장치에 따라 재정의되도록 설계됨

- 호출어 감지 결과를 텍스트로 기록하는 간단한 기본 구현

```cpp
// The default implementation writes out the name of the recognized command
// to the error console. Real applications will want to take some custom
// action instead, and should implement their own versions of this function.
void RespondToCommand(tflite::ErrorReporter* error_reporter,
                      int32_t current_time, const char* found_command,
                      uint8_t score, bool is_new_command) {
  if (is_new_command) {
    TF_LITE_REPORT_ERROR(error_reporter, "Heard %s (%d) @%dms", found_command,
                         score, current_time);
  }
}
```

#### 명령 응답기 테스트

- 테스트 코드
  - command_responder_test.cc
  - https://github.com/yunho0130/tensorflow-lite/blob/master/tensorflow/lite/micro/examples/micro_speech/command_responder_test.cc

```cpp
TF_LITE_MICRO_TEST(TestCallability) {
  tflite::MicroErrorReporter micro_error_reporter;
  tflite::ErrorReporter* error_reporter = &micro_error_reporter;

  // This will have external side-effects (like printing to the debug console
  // or lighting an LED) that are hard to observe, so the most we can do is
  // make sure the call doesn't crash.
  RespondToCommand(error_reporter, 0, "foo", 0, true);
}
```

- 테스트 실행

In [10]:
%cd /content/tensorflow-lite/

! make -f tensorflow/lite/micro/tools/make/Makefile test_command_responder_test

/content/tensorflow-lite
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/micro_speech/command_responder_test.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/micro_speech/command_responder_test.o
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/micro_speech/command_responder.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/micro_speech/command_responder.o
g++

## 7.4 호출어 감지

#### 주요 코드
- main_functions.h / main_functions.cc
  - 프로그램의 핵심인 setup(), loop() 함수를 정의
  - micro_speech_test.cc에서 본 것과 유사한 형태

- main.cc
  - 여기에 정의된 main() 함수에서 setup(), loop() 함수 호출



#### 초기화 및 setup() 함수

```cpp
// Globals, used for compatibility with Arduino-style sketches.
namespace {
tflite::ErrorReporter* error_reporter = nullptr;
const tflite::Model* model = nullptr;
tflite::MicroInterpreter* interpreter = nullptr;
TfLiteTensor* model_input = nullptr;
FeatureProvider* feature_provider = nullptr;
RecognizeCommands* recognizer = nullptr;
int32_t previous_time = 0;

// Create an area of memory to use for input, output, and intermediate arrays.
// The size of this will depend on the model you're using, and may need to be
// determined by experimentation.
constexpr int kTensorArenaSize = 10 * 1024;
uint8_t tensor_arena[kTensorArenaSize];
uint8_t feature_buffer[kFeatureElementCount];
uint8_t* model_input_buffer = nullptr;
}  // namespace
```

- 전역 변수 선언
  - 모델, 인터프리터, 특징 추출기, 명령 인식기 객체
  - 텐서 아레나, 특징 데이터를 저장할 feature_buffer 등

```cpp
// The name of this function is important for Arduino compatibility.
void setup() {
  // Set up logging. Google style is to avoid globals or statics because of
  // lifetime uncertainty, but since this has a trivial destructor it's okay.
  // NOLINTNEXTLINE(runtime-global-variables)
  static tflite::MicroErrorReporter micro_error_reporter;
  error_reporter = &micro_error_reporter;

  // Map the model into a usable data structure. This doesn't involve any
  // copying or parsing, it's a very lightweight operation.
  model = tflite::GetModel(g_tiny_conv_micro_features_model_data);
  if (model->version() != TFLITE_SCHEMA_VERSION) {
    TF_LITE_REPORT_ERROR(error_reporter,
                         "Model provided is schema version %d not equal "
                         "to supported version %d.",
                         model->version(), TFLITE_SCHEMA_VERSION);
    return;
  }

  // Pull in only the operation implementations we need.
  // This relies on a complete list of all the ops needed by this graph.
  // An easier approach is to just use the AllOpsResolver, but this will
  // incur some penalty in code space for op implementations that are not
  // needed by this graph.
  //
  // tflite::ops::micro::AllOpsResolver resolver;
  // NOLINTNEXTLINE(runtime-global-variables)
  static tflite::MicroOpResolver<3> micro_op_resolver;
  micro_op_resolver.AddBuiltin(
      tflite::BuiltinOperator_DEPTHWISE_CONV_2D,
      tflite::ops::micro::Register_DEPTHWISE_CONV_2D());
  micro_op_resolver.AddBuiltin(tflite::BuiltinOperator_FULLY_CONNECTED,
                               tflite::ops::micro::Register_FULLY_CONNECTED());
  micro_op_resolver.AddBuiltin(tflite::BuiltinOperator_SOFTMAX,
                               tflite::ops::micro::Register_SOFTMAX());

  // Build an interpreter to run the model with.
  static tflite::MicroInterpreter static_interpreter(
      model, micro_op_resolver, tensor_arena, kTensorArenaSize, error_reporter);
  interpreter = &static_interpreter;

  // Allocate memory from the tensor_arena for the model's tensors.
  TfLiteStatus allocate_status = interpreter->AllocateTensors();
  if (allocate_status != kTfLiteOk) {
    TF_LITE_REPORT_ERROR(error_reporter, "AllocateTensors() failed");
    return;
  }
```

- MicroErrorReporter 객체 생성
- Model 객체 생성
- 모델 실행에 필요한 연산을 로드하기 위해 MicroOpResolver 객체 생성
  - 2D Depthwise conv
  - Fully connected
  - Softmax
- MicroInterpreter 객체 생성
- 모델 실행을 위해 사용할 텐서 메모리 할당

```cpp
  // Get information about the memory area to use for the model's input.
  model_input = interpreter->input(0);
  if ((model_input->dims->size != 4) || (model_input->dims->data[0] != 1) ||
      (model_input->dims->data[1] != kFeatureSliceCount) ||
      (model_input->dims->data[2] != kFeatureSliceSize) ||
      (model_input->type != kTfLiteUInt8)) {
    TF_LITE_REPORT_ERROR(error_reporter,
                         "Bad input tensor parameters in model");
    return;
  }
  
  model_input_buffer = model_input->data.uint8;
```

- 입력 텐서의 형태와 타입이 올바른지 확인

```cpp
  // Prepare to access the audio spectrograms from a microphone or other source
  // that will provide the inputs to the neural network.
  // NOLINTNEXTLINE(runtime-global-variables)
  static FeatureProvider static_feature_provider(kFeatureElementCount,
                                                 feature_buffer);
  feature_provider = &static_feature_provider;

  static RecognizeCommands static_recognizer(error_reporter);
  recognizer = &static_recognizer;

  previous_time = 0;
```

- 특징 추출기 (FeatureProvider) 객체 생성
- 명령 인식기 (RecognizeCommands) 객체 생성
- 시작 시간 초기화 (previous_time)

#### loop() 함수

```cpp
// The name of this function is important for Arduino compatibility.
void loop() {
  // Fetch the spectrogram for the current time.
  const int32_t current_time = LatestAudioTimestamp();

  int how_many_new_slices = 0;

  TfLiteStatus feature_status = feature_provider->PopulateFeatureData(
      error_reporter, previous_time, current_time, &how_many_new_slices);

  if (feature_status != kTfLiteOk) {
    TF_LITE_REPORT_ERROR(error_reporter, "Feature generation failed");
    return;
  }

  previous_time = current_time;
  
  // If no new audio samples have been received since last time, don't bother
  // running the network model.
  if (how_many_new_slices == 0) {
    return;
  }
```

- 특징 추출기를 이용하여 스펙트로그램 생성

```cpp
  // Copy feature buffer to input tensor
  for (int i = 0; i < kFeatureElementCount; i++) {
    model_input_buffer[i] = feature_buffer[i];
  }
```

- 추출된 특징 데이터(스펙트로그램)를 저장한 feature_buffer 배열의 값을 입력 텐서에 복사

```cpp
  // Run the model on the spectrogram input and make sure it succeeds.
  TfLiteStatus invoke_status = interpreter->Invoke();
  if (invoke_status != kTfLiteOk) {
    TF_LITE_REPORT_ERROR(error_reporter, "Invoke failed");
    return;
  }
```

- 인터프리터를 실행하여 모델의 출력 결과를 생성

```cpp
  // Obtain a pointer to the output tensor
  TfLiteTensor* output = interpreter->output(0);

  // Determine whether a command was recognized based on the output of inference
  const char* found_command = nullptr;
  uint8_t score = 0;
  bool is_new_command = false;

  TfLiteStatus process_status = recognizer->ProcessLatestResults(
      output, current_time, &found_command, &score, &is_new_command);
      
  if (process_status != kTfLiteOk) {
    TF_LITE_REPORT_ERROR(error_reporter,
                         "RecognizeCommands::ProcessLatestResults() failed");
    return;
  }
```

- 모델의 출력 텐서 데이터를 이용하여 호출어 감지 수행
  - 모델 출력 텐서에는 각 클래스 레이블의 확률이 기록됨
  - 이를 ProcessLatestResults() 함수로 전달하여 처리

```cpp
  // Do something based on the recognized command. The default implementation
  // just prints to the error console, but you should replace this with your
  // own function for a real application.
  RespondToCommand(error_reporter, current_time, found_command, score,
                   is_new_command);
```

- 명령 응답기의 RespondToCommand() 메소드를 호출하여 호출어 감지 결과를 출력

#### 애플리케이션 실행

- 애플리케이션 빌드 수행


In [11]:
! make -f tensorflow/lite/micro/tools/make/Makefile micro_speech

g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/micro_speech/main.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/micro_speech/main.o
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON -I. -Itensorflow/lite/micro/tools/make/downloads/ -Itensorflow/lite/micro/tools/make/downloads/gemmlowp -Itensorflow/lite/micro/tools/make/downloads/flatbuffers/include -Itensorflow/lite/micro/tools/make/downloads/kissfft -c tensorflow/lite/micro/examples/micro_speech/main_functions.cc -o tensorflow/lite/micro/tools/make/gen/linux_x86_64/obj/tensorflow/lite/micro/examples/micro_speech/main_functions.o
g++ -std=c++11 -DTF_LITE_STATIC_MEMORY -O3 -DTF_LITE_DISABLE_X86_NEON 

- 빌드된 애플리케이션 실행 파일 위치 (colab에서 빌드한 경우)
  - tensorflow/lite/micro/tools/make/gen/linux_x86_64/bin/micro_speech

  - colab에서는 오디오 데이터를 추출할 수 있는 환경이 안 되기 때문에 실행하여 결과를 확인해 볼 수는 없음

- 출력 예제

Heard yes(201) @4056ms


Heard no(205) @6448ms


Heard unknown(201) @13696ms


Heard yes(205) @15000ms


 - 감지된 단어
 - 점수 (괄호 안): 명령 인식기는 점수가 200보다 큰 경우에만 일치하는 것으로 간주하므로 모두 200 이상
 - 프로그램이 시작된 이후 경과한 시간 (밀리초)

## 7.5 마이크로컨트롤러에 배포하기

- 장치별 버전
  - 모든 장치에는 자체적인 오디오 캡처 메커니즘이 있으므로 각 장치마다 audio_provider.cc를 별도로 구현해야 함
  - 출력도 마찬가지므로 command_responder.cc도 장치별 버전이 필요함


- audio_provider.cc의 아두이노 버전: arduino_audio_provider.cpp

- command_responder.cc의 아두이노 버전: arduino_command_responder.cpp

- 아두이노 나노 33 BLE 센스를 기준으로 구현된 코드이고 만약 다른 아두이노 보드를 사용하고 별도의 마이크를 연결하려면 audio_provider.cc를 직접 바꿔서 구현해야 함


#### 아두이노의 명령 반응

- 인식된 결과에 따라 LED를 약 3초간 켬
 - yes가 인식되면 녹색
 - no가 인식되면 빨간색
 - unknown인 경우 파란색

- arduino_command_responder.cpp 주요 코드





```cpp
// Toggles the built-in LED every inference, and lights a colored LED depending
// on which word was detected.
void RespondToCommand(tflite::ErrorReporter* error_reporter,
                      int32_t current_time, const char* found_command,
                      uint8_t score, bool is_new_command) {

  static bool is_initialized = false;

  if (!is_initialized) {
    pinMode(LED_BUILTIN, OUTPUT);
    // Pins for the built-in RGB LEDs on the Arduino Nano 33 BLE Sense
    pinMode(LEDR, OUTPUT);
    pinMode(LEDG, OUTPUT);
    pinMode(LEDB, OUTPUT);
    is_initialized = true;
  }
  static int32_t last_command_time = 0;
  static int count = 0;
  static int certainty = 220;
```

- 내장 LED 핀을 출력 모드로 설정
  - is_initialized라는 static bool 변수를 이용해 한 번만 실행되는 if 문 내에서 작업 수행


```cpp
  if (is_new_command) {
    error_reporter->Report("Heard %s (%d) @%dms", found_command, score,
                           current_time);
    // If we hear a command, light up the appropriate LED.
    // Note: The RGB LEDs on the Arduino Nano 33 BLE
    // Sense are on when the pin is LOW, off when HIGH.
    if (found_command[0] == 'y') {
      last_command_time = current_time;
      digitalWrite(LEDG, LOW);  // Green for yes
    }

    if (found_command[0] == 'n') {
      last_command_time = current_time;
      digitalWrite(LEDR, LOW);  // Red for no
    }

    if (found_command[0] == 'u') {
      last_command_time = current_time;
      digitalWrite(LEDB, LOW);  // Blue for unknown
    }
  }
```

- is_new_command 변수가 true이면 새로운 호출어를 인식한 것이므로 error_reporter 객체를 이용해 결과 출력

- 인식 결과를 저장한 found_command 문자 배열의 첫 문자를 확인하여 "yes", "no", "unknown" 결과에 따라 해당 LED를 켬
  - LOW가 on이고 HIGH는 off

```cpp
  // If last_command_time is non-zero but was >3 seconds ago, zero it
  // and switch off the LED.
  if (last_command_time != 0) {
    if (last_command_time < (current_time - 3000)) {
      last_command_time = 0;
      digitalWrite(LED_BUILTIN, LOW);
      digitalWrite(LEDR, HIGH);
      digitalWrite(LEDG, HIGH);
      digitalWrite(LEDB, HIGH);
    }
    // If it is non-zero but <3 seconds ago, do nothing.
    return;
  }
```

- 3초 후에 LED를 끄는 동작을 구현

```cpp
  // Otherwise, toggle the LED every time an inference is performed.
  ++count;
  if (count & 1) {
    digitalWrite(LED_BUILTIN, HIGH);
  } else {
    digitalWrite(LED_BUILTIN, LOW);
  }
```

- 추론이 수행될 때마다 BUILTIN LED (주황색 LED) 점등
  - 추론 횟수를 저장하는 count 변수
  - count 변수와 1에 대해 AND 연산 수행
  - count가 홀수이면 AND 결과는 1, 짝수이면 AND 결과는 0이 됨
  - 홀수일 때 LED를 끄고 짝수일 때 LED를 켬

#### 아두이노에서 예제 실행

- 예제 로드
  - 파일 메뉴 >> 예제 >> Arduino_TensorFlowLite >> micro_speech 선택
  - 첫번째 열리는 탭인 micro_speech 파일이 main_functions.cc에 해당

- USB 케이블로 아두이노를 컴퓨터에 연결

- Arduino IDE에서 해당 보드(Arduino Nano 33 BLE) 선택
  - 툴 메뉴 >> 보드 (Board)

- 포트 선택
  - 툴 메뉴 >> 포트 (Port)

- Arduino IDE에서 업로드 버튼(오른쪽 방향 화살표) 클릭

- "yes", "no"라고 말하고 그것이 인식되면 LED가 서로 다른 빛으로 점등


## 7.6 실습 과제

- 아두이노 IDE에서 소스 파일을 편집하여 커스텀 버전 애플리케이션을 만들어서 실행해보자

 - yes라고 할 때만 파란색 LED가 켜지고 다른 경우는 안 켜지게 만든다

 - 애플리케이션이 모스 부호처럼 yes와 no의 특정 순서 조합에 응답하도록 만든다. 예를 들어 yes no yes가 인식되면 녹색 LED를 켜고, no no no가 인식되면 빨간 LED를 켬
